<a href="https://colab.research.google.com/github/richardOlson/nlp__tranformers/blob/main/squad_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# looking at the squad dataset
# getting some of the imports
import requests
import json
import pandas as pd


In [2]:
files = ['train-v2.0.json', 'dev-v2.0.json']

In [3]:
url = "https://rajpurkar.github.io/SQuAD-explorer/dataset/"

In [4]:
# now getting the files 
for file in files:
  res = requests.get(url + file)
  if res.status_code == 200:
    
    with open(file, mode="wb") as f:
     # writing as a chunks 
     for chunk in res.iter_content(chunk_size=50):
       f.write(chunk)



In [5]:
# reading in one of the files to be used with json
with open("dev-v2.0.json", mode="rb")as f:
  dev = json.load(f)

with open("train-v2.0.json", mode="rb")as f:
  train = json.load(f)

In [6]:
train.keys()

dict_keys(['version', 'data'])

In [ ]:
train["data"][0]["title"]

'Beyoncé'

In [ ]:
type(train["data"][0]["paragraphs"])

list

In [8]:
squad_list = []

Getting the data into a format that can be used

In [9]:
# now getting the info out of the train
for paragraph_dict in train["data"]:
  for context_with_qa_dict in paragraph_dict["paragraphs"]:
    context = context_with_qa_dict["context"]
    # doing the looping through the question and the answers dictionaries
    for qa_pair_dict in context_with_qa_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
          answer = qa_pair_dict['answers'][0]['text']
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer = qa_pair_dict["plausible_answers"][0]["text"]
      else:
        answer = None
      # now making the dictionary that will be added to the list 
      squad_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})
        
      


In [10]:
len(squad_list)

130319

In [ ]:
# creating the dataframe
df = pd.DataFrame(squad_list)

In [ ]:
print(df.shape)
df.head()

(130319, 3)


,context,question,answer
0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,in the late 1990s
1,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,singing and dancing
2,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,2003
3,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,"Houston, Texas"
4,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,late 1990s


In [ ]:
df.loc[0, "context"]

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [11]:
# now will save the data into a json type object
with open("train.json", mode="w", )as f:
  json.dump(squad_list, f)

In [13]:
dev_list = []

In [14]:
# doing the dev to make it so that it is in the same format as the 
# train.
for paragraph_dict in dev["data"]:
  for each_context_dict in paragraph_dict["paragraphs"]:
    context = each_context_dict["context"]
    for qa_pair_dict in each_context_dict["qas"]:
      if "answers" in qa_pair_dict and len(qa_pair_dict["answers"]) > 0:
        # doing the looping through the answers
        answer_list = qa_pair_dict["answers"]
      elif "plausible_answers" in qa_pair_dict and len(qa_pair_dict["plausible_answers"]) > 0:
        answer_list = qa_pair_dict["plausible_answers"]
      else:
        answer_list = []
      # now doing the making of the list  
      answer = [item["text"] for item in answer_list]
      # removing any duplicates
      answer = list(set(answer))
      
      # adding to the dev list
      dev_list.append({"context": context, "question": qa_pair_dict["question"], "answer": answer})


In [15]:
len(dev_list)

11873

In [ ]:
# building a dataframe of the dev_list
dev_dataframe = pd.DataFrame(dev_list)
dev_dataframe.head()

,context,question,answer
0,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,[France]
1,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,"[in the 10th and 11th centuries, 10th and 11th..."
2,The Normans (Norman: Nourmands; French: Norman...,From which countries did the Norse originate?,"[Denmark, Iceland and Norway]"
3,The Normans (Norman: Nourmands; French: Norman...,Who was the Norse leader?,[Rollo]
4,The Normans (Norman: Nourmands; French: Norman...,What century did the Normans first gain their ...,"[the first half of the 10th century, 10th cent..."


In [16]:
# making a json file of the new formatted dev
with open("dev.json", mode="w")as f:
  json.dump(dev_list, f)

In [17]:
# now making the first model 
! pip install transformers -q
import transformers
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import pipeline

     |████████████████████████████████| 2.6 MB 7.6 MB/s 
     |████████████████████████████████| 636 kB 51.2 MB/s 
     |████████████████████████████████| 895 kB 48.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.6 MB/s 


In [18]:
# making the model
model = BertForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")
tokenizer = BertTokenizer.from_pretrained("deepset/bert-base-cased-squad2")

Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

In [19]:
# going to do it two ways -- one with using the pipelines and the other without the pipeline in 
# transformers

model.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [22]:
# the span of the questions that we will send into the tokenizer
my_span = squad_list[:5]
my_span

[{'answer': 'in the late 1990s',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
  'question': 'When did Beyonce start becoming popular?'},
 {'answer': 'singing and dancing',
  'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress

In [23]:
# putting the 
questions = [item["question"] for item in my_span]
contexts = [item["context"] for item in my_span]

In [27]:
questions

['When did Beyonce start becoming popular?',
 'What areas did Beyonce compete in when she was growing up?',
 "When did Beyonce leave Destiny's Child and become a solo singer?",
 'In what city and state did Beyonce  grow up? ',
 'In which decade did Beyonce become famous?']

In [35]:
# doing some encoding of the text
input_ids = tokenizer.encode(questions[0], contexts[0])
t = len(input_ids)
print(f"The length of the input_ids is: {t}")

The length of the input_ids is: 173


In [31]:
tokenizer.sep_token_id

102

In [ ]:
# making a function that will be doing the encoding and print out the 
# question and the answers
def answer_question(question:[list, str], context[list, str]):
  # single context is a flag to wheather their is just on context for all 
  # the questions
  single_context = False
  imput_ids = None

  if isinstance(question, list):

    if isinstance(context, str):
      single_context = True
  else:
    question = [question]
  # running the for loop
  for i, q in enumerate(question):
    if single_context:
      input_ids = tokenizer.encode(q, context)
    else:
      input_ids = tokenizer.encode(q, context[i])
      # running them in the model
      output = model(input_ids)
      # getting the span

  